In [22]:
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
from astromodels import Powerlaw, Line, PointSource, SpectralComponent
import astropy.time as at
from datetime import datetime
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
import os
from MultinestClusterFit import powerlaw_binned_spectrum
import pickle

In [33]:
data_path = "crab_data/0374"

In [34]:
source_ra, source_dec = 10, -40
source_piv = 100.
source_Ks = [0.5e-4, 2e-4, 8e-4]
source_indices = [-0.5, -2, -8]

In [35]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_Ks, source_indices), f)


In [36]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [37]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [38]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [39]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [40]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [41]:
for i in range(len(source_Ks)):
    for j in range(len(source_indices)):
        
        temp_path = f"{data_path_d}/{i}_{j}"        
        
        if not os.path.exists(temp_path):
            os.mkdir(temp_path)
            
        os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
        os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

                
        hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
        hdu.writeto(f"{temp_path}/dead_time.fits")
        
        source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_Ks[i], source_indices[j])
        
        total_counts = background_counts.copy()
        total_counts["COUNTS"] += source_counts
                
        hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
        hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [ ]:
# second source

In [2]:
data_path = "crab_data/0374"
# data_path = "crab_data/1380"

In [3]:
# primary

source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

# secondary
s_source_decs = [-40, -40.5, -41.5, -45, -65, -85]
s_source_Ks = [0.1e-4, 0.3e-4, 1e-4]
s_source_index = -3

In [4]:
pointing_index = 1
# pointing_index = 4


data_path_d = "simulated_data/0374_const_bkg_sec_source_i_3"
# data_path_d = "simulated_data/1380_const_bkg_sec_source"


if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, s_source_decs, s_source_Ks, s_source_index), f)


In [5]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [6]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [7]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [8]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [12]:
s_resp_mats = []

for i in range(len(s_source_decs)):
    t_resp_mats = []
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        time = time_start[p_i]
        dets = get_live_dets(time=time, event_types=["single"])
        
        rmfs = []
        for d in dets:
            rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
            
        sds = np.empty(0)
        for d in range(len(dets)):
            sd = SPIDRM(rmfs[d], source_ra, s_source_decs[i])
            sds = np.append(sds, sd.matrix.T)
        t_resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))
    s_resp_mats.append(t_resp_mats)

In [13]:
def calc_count_rates_sec(resp_mats, ra, dec, piv, K, index, s_resp_mats, s_dec, s_K, s_index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    s_pl = Powerlaw()
    s_pl.piv = piv
    s_pl.K = s_K
    s_pl.index = s_index
    component1 = SpectralComponent("pl", shape=s_pl)
    s_source = PointSource("Test", ra=ra, dec=s_dec, components=[component1])
    
    s_spec = s_source(emod)
    s_spec_binned = powerlaw_binned_spectrum(emod, s_spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1))
    
    s_source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1))
        
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
                
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
                        
        s_resp_mat = s_resp_mats[p_i]
        
        s_count_rates = np.dot(s_spec_binned, s_resp_mat)
                    
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            s_source_counts[index,:] = np.random.poisson(s_count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
            
    total_source_counts = source_counts + s_source_counts
    
    return total_source_counts

In [18]:
for i in range(len(s_source_decs)):
    for j in range(len(s_source_Ks)):
        
        temp_path = f"{data_path_d}/{i}_{j}"        
        
        if not os.path.exists(temp_path):
            os.mkdir(temp_path)
            
        os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
        os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

                
        hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
        hdu.writeto(f"{temp_path}/dead_time.fits")
        
        source_counts = calc_count_rates_sec(resp_mats, source_ra, source_dec, source_piv, source_K, source_index, s_resp_mats[i], s_source_decs[i], s_source_Ks[j], s_source_index)
        
        total_counts = background_counts.copy()
        total_counts["COUNTS"] = source_counts + total_counts["COUNTS"]
                
        hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
        hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# energy ranges

data_path = "crab_data/0374"

In [4]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

energy_ranges = [(None, 80),
                 (None, 200),
                 (None, 1000),
                 (None, None),
                 (80, None),
                 (200, None),
                 (1000, None)]

In [5]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_energy_ranges"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, energy_ranges), f)


In [6]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [7]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [8]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [9]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [9]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [10]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# identical repeats

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

In [5]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_identical"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index), f)


In [6]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [7]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [8]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [9]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [10]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [11]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# number of energy bins

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

bin_number = [5, 25, 125, 600]

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_number_of_e_bins"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, bin_number), f)


In [5]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [6]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [7]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [8]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [ ]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [ ]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# length of SCWs

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

time_fraction = [1, 0.3, 0.1]

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_scaled_time_or_pairs"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, time_fraction), f)


In [6]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [7]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [8]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])

b_cs = []

for frac in range(len(time_fraction)):

    background_counts = t.copy()

    for i in range(int(len(background_counts) / 85)):
        if i == pointing_index:
            continue
        else:
            background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
            
    background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"] * time_fraction[frac])
    b_cs.append(background_counts)

In [9]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [27]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index, time_elapsed):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [28]:
for i in range(len(time_fraction)):
        
    temp_path = f"{data_path_d}/t_{i}"        

    if not os.path.exists(temp_path):
        os.mkdir(temp_path)
        
    os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
    os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

    temp_time = updated_time.copy()
    temp_time["LIVETIME"] *= time_fraction[i]
    hdu = fits.BinTableHDU(data=temp_time, name="SPI.-OBS.-DTI") # is all of this correct?
    hdu.writeto(f"{temp_path}/dead_time.fits")

    source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index, time_elapsed*time_fraction[i])

    total_counts = b_cs[i].copy()
    total_counts["COUNTS"] += source_counts
            
    hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
    hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# cluster sizes

data_path = "crab_data/0374"

In [3]:
source_ra, source_dec = 10, -40
source_piv = 200.
source_K = 1e-4
source_index = -2

cluster_sizes = [2, 3, 4]

In [4]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_bkg_cluster_sizes"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index, cluster_sizes), f)


In [13]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]

In [14]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [15]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

for i in range(int(len(background_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        background_counts[i*85 : (i+1)*85] = background_counts[pointing_index*85 : (pointing_index+1)*85]
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [16]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [ ]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [ ]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [2]:
# const const bkg

In [26]:
data_path = "crab_data/0374"

In [27]:
source_ra, source_dec = 10, -40
source_piv = 100.
source_K = 8e-4
source_index = -0.5

In [28]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_const_bkg_200"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_piv, source_K, source_index), f)


In [29]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
time_elapsed = np.full(time_elapsed.shape, 3000.)
    
updated_time = t.copy()
updated_time["LIVETIME"] = time_elapsed

In [30]:
updated_time

DEADRATIO,LIVETIME
float32,float64
0.8494475,3000.0
0.84993005,3000.0
1.0,3000.0
0.8495635,3000.0
0.84933347,3000.0
0.8495878,3000.0
0.84968084,3000.0
0.8505347,3000.0
0.85012025,3000.0


In [31]:
time_elapsed

array([3000., 3000., 3000., ..., 3000., 3000., 3000.])

In [32]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [33]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

background_counts["COUNTS"] = np.full(background_counts["COUNTS"].shape, 200.)
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [34]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [35]:
def calc_count_rates(resp_mats, ra, dec, piv, K, index):
    pl = Powerlaw()
    pl.piv = piv
    pl.K = K
    pl.index = index
    component1 = SpectralComponent("pl", shape=pl)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = powerlaw_binned_spectrum(emod, spec)
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [25]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_piv, source_K, source_index)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [ ]:
# const const bkg lin source

In [10]:
data_path = "crab_data/0374"

In [11]:
source_ra, source_dec = 10, -40
source_slope = 8e-6

In [12]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_const_bkg_20_lin_source"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_slope), f)


In [13]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
time_elapsed = np.full(time_elapsed.shape, 3000.)
    
updated_time = t.copy()
updated_time["LIVETIME"] = time_elapsed

In [14]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [15]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()

background_counts["COUNTS"] = np.full(background_counts["COUNTS"].shape, 20.)
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [16]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)



WARNING NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.



In [19]:
def calc_count_rates(resp_mats, ra, dec, b):
    s = Line()
    s.a = 0
    s.a.free = False
    s.b = b
    component1 = SpectralComponent("line", shape=s)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [21]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_slope)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

In [ ]:
# const const bkg lin source pre_binned

In [56]:
data_path = "crab_data/0374"

In [57]:
source_ra, source_dec = 10, -40
source_slope = 8e-6

In [58]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_const_bkg_20_lin_source_pre_binned_log"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_slope), f)


In [59]:
from RebinningFunctions import *

In [60]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    ebin_table = t
    old_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    

ebin_table = ebin_table[:len(energy_bins)-1]
ebin_table["E_MIN"] = energy_bins[:-1]
ebin_table["E_MAX"] = energy_bins[1:]
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
time_elapsed = np.full(time_elapsed.shape, 3000.)
    
updated_time = t.copy()
updated_time["LIVETIME"] = time_elapsed

In [61]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [68]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()
# background_counts["COUNTS"] = background_counts["COUNTS"][:,:len(energy_bins)-1]


background_counts["COUNTS"] = np.full(background_counts["COUNTS"].shape, 20.)
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [69]:
energy_bins, background_counts["COUNTS"] = exp_binning_function_for_x_number_of_bins(125)(old_energy_bins, background_counts["COUNTS"], (None, None))


In [70]:
energy_bins

array([  18. ,   18.5,   19.5,   20. ,   21. ,   21.5,   22.5,   23.5,
         24.5,   25.5,   26. ,   27. ,   28.5,   29.5,   30.5,   31.5,
         33. ,   34. ,   35.5,   37. ,   38. ,   39.5,   41. ,   43. ,
         44.5,   46. ,   48. ,   50. ,   51.5,   53.5,   56. ,   58. ,
         60. ,   62.5,   65. ,   67.5,   70. ,   72.5,   75.5,   78.5,
         81.5,   84.5,   87.5,   91. ,   94.5,   98. ,  102. ,  106. ,
        110. ,  114. ,  118.5,  123. ,  127.5,  132.5,  137.5,  143. ,
        148.5,  154. ,  160. ,  166.5,  172.5,  179.5,  186. ,  193.5,
        201. ,  208.5,  216.5,  225. ,  233.5,  242.5,  251.5,  261.5,
        271.5,  282. ,  292.5,  304. ,  315.5,  327.5,  340.5,  353.5,
        367. ,  381. ,  395.5,  411. ,  426.5,  443. ,  460. ,  477.5,
        496. ,  515. ,  535. ,  555.5,  576.5,  599. ,  622. ,  645.5,
        670.5,  696.5,  723. ,  751. ,  779.5,  809.5,  840.5,  873. ,
        906.5,  941.5,  977.5, 1015. , 1054. , 1094.5, 1136.5, 1180. ,
      

In [71]:
background_counts["COUNTS"][0]

array([  15,   43,   27,   39,   13,   54,   38,   35,   36,   21,   42,
         68,   41,   52,   42,   61,   33,   50,   72,   40,   61,   65,
         70,   57,   48,   68,   87,   60,   83,   92,   78,   83,  111,
         77,   87,   70,  108,  125,  129,  130,  122,  128,  153,  145,
        141,  152,  156,  165,  148,  202,  172,  183,  188,  172,  208,
        217,  224,  238,  268,  240,  291,  276,  287,  321,  307,  342,
        335,  326,  357,  359,  388,  397,  381,  426,  461,  467,  487,
        534,  521,  527,  530,  576,  626,  589,  683,  688,  665,  714,
        766,  802,  850,  791,  870,  914,  937, 1036, 1035,  991, 1134,
       1177, 1186, 1229, 1304, 1398, 1396, 1443, 1553, 1640, 1549, 1669,
       1764, 1825, 1908, 2000, 2038, 2147, 2309, 2274, 2272, 2419, 2566,
       2656, 2732, 2808, 2960])

In [72]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [73]:
def calc_count_rates(resp_mats, ra, dec, b):
    s = Line()
    s.a = 0
    s.a.free = False
    s.b = b
    component1 = SpectralComponent("line", shape=s)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [74]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
# os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_slope)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

hdu = fits.BinTableHDU(data=ebin_table, name="SPI.-OBS.-EB")
hdu.writeto(f"{temp_path}/energy_boundaries.fits")

In [75]:
# const const bkg lin source pre_binned pre_cut

In [76]:
data_path = "crab_data/0374"

In [77]:
source_ra, source_dec = 10, -40
source_slope = 8e-6

In [78]:
pointing_index = 1

data_path_d = "simulated_data/0374_const_const_bkg_20_lin_source_pre_binned_log_cut"

if not os.path.exists(f"{data_path_d}"):
    os.mkdir(f"{data_path_d}")
    
with open(f"./{data_path_d}/source_params.pickle", "wb") as f:
    pickle.dump((source_ra, source_dec, source_slope), f)


In [79]:
from RebinningFunctions import *

In [80]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    ebin_table = t
    old_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    

ebin_table = ebin_table[:len(energy_bins)-1]
ebin_table["E_MIN"] = energy_bins[:-1]
ebin_table["E_MAX"] = energy_bins[1:]
    
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])
    
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
time_elapsed = np.full(time_elapsed.shape, 3000.)
    
updated_time = t.copy()
updated_time["LIVETIME"] = time_elapsed

In [81]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
# skip_pointing[0] = True

In [82]:
# Background

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
background_counts = t.copy()
# background_counts["COUNTS"] = background_counts["COUNTS"][:,:len(energy_bins)-1]


background_counts["COUNTS"] = np.full(background_counts["COUNTS"].shape, 20.)
        
background_counts["COUNTS"] = np.random.poisson(background_counts["COUNTS"])

In [83]:
energy_bins, background_counts["COUNTS"] = exp_binning_function_for_x_number_of_bins(125)(old_energy_bins, background_counts["COUNTS"], (50., 2000.))


In [84]:
energy_bins

array([  50. ,   51.5,   53. ,   54.5,   56.5,   58. ,   59.5,   61.5,
         63.5,   65. ,   67. ,   69. ,   71. ,   73.5,   75.5,   78. ,
         80. ,   82.5,   85. ,   87.5,   90. ,   93. ,   95.5,   98.5,
        101.5,  104.5,  107.5,  111. ,  114. ,  117.5,  121. ,  125. ,
        128.5,  132.5,  136.5,  140.5,  144.5,  149. ,  153.5,  158. ,
        163. ,  167.5,  172.5,  178. ,  183. ,  188.5,  194.5,  200. ,
        206. ,  212.5,  218.5,  225. ,  232. ,  239. ,  246. ,  253.5,
        261. ,  269. ,  277. ,  285. ,  293.5,  302.5,  311.5,  321. ,
        330.5,  340.5,  350.5,  361. ,  372. ,  383. ,  394.5,  406.5,
        418.5,  431. ,  444. ,  457.5,  471. ,  485. ,  499.5,  514.5,
        530. ,  546. ,  562. ,  579. ,  596.5,  614.5,  632.5,  651.5,
        671. ,  691.5,  712. ,  733.5,  755. ,  778. ,  801. ,  825. ,
        850. ,  875.5,  901.5,  928.5,  956.5,  985. , 1014.5, 1045. ,
       1076. , 1108.5, 1141.5, 1176. , 1211. , 1247.5, 1284.5, 1323. ,
      

In [85]:
background_counts["COUNTS"][0]

array([  62,   70,   58,   73,   56,   67,   76,   81,   68,   82,   78,
         80,   84,   75,  109,   90,  105,   92,  106,   95,  128,  108,
        133,  100,  136,  128,  140,  132,  161,  140,  140,  150,  191,
        137,  171,  177,  195,  171,  192,  180,  182,  155,  227,  180,
        226,  244,  209,  237,  264,  234,  259,  292,  273,  254,  296,
        333,  299,  313,  317,  324,  386,  390,  384,  397,  406,  423,
        421,  461,  420,  469,  512,  472,  489,  530,  543,  550,  532,
        567,  623,  619,  607,  622,  701,  686,  702,  681,  713,  794,
        809,  805,  833,  831,  895,  916,  939,  993, 1030, 1041, 1117,
       1107, 1141, 1173, 1191, 1288, 1319, 1393, 1346, 1413, 1462, 1456,
       1549, 1560, 1731, 1730, 1836, 1743, 1792, 1858, 2017, 2017, 2014,
       2111, 2205, 2190, 2358])

In [86]:
assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)


resp_mats = []
emod = np.geomspace(10, 3000, 50)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], source_ra, source_dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mats.append(sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1)))

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [ ]:
def calc_count_rates(resp_mats, ra, dec, b):
    s = Line()
    s.a = 0
    s.a.free = False
    s.b = b
    component1 = SpectralComponent("line", shape=s)
    source = PointSource("Test", ra=ra, dec=dec, components=[component1])
    
    spec = source(emod)
    spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2
    
    source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)
    
    for p_i, pointing in enumerate(pointings):
        if skip_pointing[p_i]:
            continue
        
        resp_mat = resp_mats[p_i]
        
        count_rates = np.dot(spec_binned, resp_mat)
        
        for d_i, d in enumerate(dets):
            index = p_i * 85 + d
            source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])
    
    return source_counts

In [ ]:

        
temp_path = f"{data_path_d}"        

if not os.path.exists(temp_path):
    os.mkdir(temp_path)
    
# os.popen(f"cp {data_path}/energy_boundaries.fits {temp_path}/energy_boundaries.fits")
os.popen(f"cp {data_path}/pointing.fits {temp_path}/pointing.fits")

        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{temp_path}/dead_time.fits")

source_counts = calc_count_rates(resp_mats, source_ra, source_dec, source_slope)

total_counts = background_counts.copy()
total_counts["COUNTS"] += source_counts
        
hdu = fits.BinTableHDU(data=total_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{temp_path}/evts_det_spec.fits")

hdu = fits.BinTableHDU(data=ebin_table, name="SPI.-OBS.-EB")
hdu.writeto(f"{temp_path}/energy_boundaries.fits")